# **Analyse Exploratoire**

In [1]:
# Bibliothèques de base
import pandas as pd
import numpy as np

# Visualisations
import seaborn as sns
import matplotlib.pyplot as plt

# Utilisés lors de l'ACP
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Servira pour faire l'ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Pour faire tests de normalité
from scipy.stats import shapiro

# Utilisé pour passer des données au log avant ANOVA
from sklearn.preprocessing import FunctionTransformer

# Utilisé pour savoir si le fichier excel d'export existe déjà
import os

# Je désactive le copy warning, en m'étant au préalable assuré que les traitements étaient corrects
# pd.options.mode.chained_assignment = None  # default='warn'

## **Importation des datasets**

In [2]:
customers = pd.read_csv("dataset\olist_customers_dataset.csv", sep=',')
geolocation = pd.read_csv("dataset\olist_geolocation_dataset.csv", sep=',')
order_items = pd.read_csv("dataset\olist_order_items_dataset.csv", sep=',')
order_payments = pd.read_csv("dataset\olist_order_payments_dataset.csv", sep=',')
order_reviews = pd.read_csv("dataset\olist_order_reviews_dataset.csv", sep=',')
orders = pd.read_csv("dataset\olist_orders_dataset.csv", sep=',')
products = pd.read_csv("dataset\olist_products_dataset.csv", sep=',')
sellers = pd.read_csv("dataset\olist_sellers_dataset.csv", sep=',')
product_category_name_translation = pd.read_csv("dataset\product_category_name_translation.csv", sep=',')

print('Importation terminée')

Importation terminée


## **Pré-traitement des datasets avant fusion**

### **Traitement de *geolocation***

Je supprime *geolocation_city* et *geolocation_state* que je ne vais pas utiliser.<br>
Je regroupe par *geolocation_zip_code_prefix* en faisant la moyen pour les coordonnées GPS.

In [3]:
print('Lignes avant traitement :', len(geolocation))
geolocation = geolocation.drop(['geolocation_city', 'geolocation_state'], axis=1)
geolocation = geolocation.groupby('geolocation_zip_code_prefix').mean()
geolocation = geolocation.reset_index() # pour remettre geolocation_zip_code_prefix en tant que colonne
print('Lignes après traitement :', len(geolocation))
print('Valeurs manquantes :', geolocation.isnull().sum().sum())

Lignes avant traitement : 1000163
Lignes après traitement : 19015
Valeurs manquantes : 0


### **Traitement de *customers***

Je fusionne avec *geolocation* pour récupérer les coordonnées GPS à partir du zip code prefix<br>
Je renomme *geolocation_lat* et *geolocation_lng* en *customer_lat* et *customer_lng*<br>
Je ne conserve au final que *customer_id*, *customer_unique_id*, *customer_lat* et *customer_lng*

In [4]:
customers = customers.merge(geolocation, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
customers.rename(columns={"geolocation_lat": "customer_lat", "geolocation_lng": "customer_lng"}, inplace=True)
customers = customers[['customer_id', 'customer_unique_id', 'customer_lat', 'customer_lng']]
# Il y a quelques valeurs manquantes dans customer_lat/customer_lng : j'impute par la moyenne
customers['customer_lat'].loc[customers['customer_lat'].isnull() == True] = customers['customer_lat'].mean()
customers['customer_lng'].loc[customers['customer_lng'].isnull() == True] = customers['customer_lng'].mean()
print('Valeurs manquantes :', customers.isnull().sum().sum())

Valeurs manquantes : 0


### **Traitement de *sellers***

Je fusionne avec *geolocation* pour récupérer les coordonnées GPS à partir du zip code prefix<br>
Je renomme *geolocation_lat* et *geolocation_lng* en *seller_lat* et *seller_lng*<br>
Je ne conserve au final que *seller_id*, *seller_lat*, et *seller_lng*

In [5]:
sellers = sellers.merge(geolocation, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
sellers.rename(columns={"geolocation_lat": "seller_lat", "geolocation_lng": "seller_lng"}, inplace=True)
sellers = sellers[['seller_id', 'seller_lat', 'seller_lng']]
# Il y a quelques valeurs manquantes dans seller_lat/seller_lng : j'impute par la moyenne
sellers['seller_lat'].loc[sellers['seller_lat'].isnull() == True] = sellers['seller_lat'].mean()
sellers['seller_lng'].loc[sellers['seller_lng'].isnull() == True] = sellers['seller_lng'].mean()
print('Valeurs manquantes :', sellers.isnull().sum().sum())

Valeurs manquantes : 0


### **Traitement de *order_items***

Je regroupe par numéro de commande (order_id), les autres colonnes sont agglomérées comme suit : 
- *order_item_id* : *max()*, ce qui donnera le nombre d'articles de la commande
- *product_id* : *first()*, je ne conserve que le premier, pas idéal mais ça ne concerne qu'un nombre très limité de commandes
- *seller_id* : *first()*, même logique que pour *product_id*
- *shipping_limit_date* : *first()*, c'est la même pour toute la commande
- *price* : *sum()*, prix total des articles
- *freight_value* : *mean()*, frais de port moyens

In [6]:
print('Lignes avant traitement :', len(order_items))
order_items = order_items.groupby('order_id').agg({'order_item_id': 'max', 'product_id': 'first', 'seller_id': 'first', 'shipping_limit_date': 'first', 'price': 'sum', 'freight_value': 'mean'})
order_items = order_items.reset_index() # pour remettre order_id en tant que colonne
print('Lignes après traitement :', len(order_items))
print('Valeurs manquantes :', order_items.isnull().sum().sum())

Lignes avant traitement : 112650
Lignes après traitement : 98666
Valeurs manquantes : 0


### **Traitement de *order_payments***

Je supprime *payment_sequential*, que je ne vais pas utiliser.<br>
Je regroupe par *order_id*, les autres colonnes sont agglomérées comme suit : 
- *payment_type* : *first()*
- *payment_installments* (paiements en plusieurs fois) : *max()*
- *payment_value* : *sum()*

In [7]:
print('Lignes avant traitement :', len(order_payments))
order_payments = order_payments.drop(['payment_sequential'], axis=1)
order_payments = order_payments.groupby('order_id').agg({'payment_type': 'first', 'payment_installments': 'max', 'payment_value': 'sum'})
order_payments = order_payments.reset_index() # pour remettre order_id en tant que colonne
print('Lignes après traitement :', len(order_payments))
print('Valeurs manquantes :', order_payments.isnull().sum().sum())

Lignes avant traitement : 103886
Lignes après traitement : 99440
Valeurs manquantes : 0


### **Traitement de *order_reviews***

Je supprime *review_id*, *review_comment_title*, *review_comment_message*, *review_creation_date* et *review_answer_timestamp*, que je n'utiliserai pas.<br>
Je regroupe par *order_id* avec *mean()* sur *review_score* (il arrive parfois qu'il y ait plusieurs avis pour une même commande)

In [8]:
print('Lignes avant traitement :', len(order_reviews))
order_reviews = order_reviews.drop(['review_id', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'], axis=1)
order_reviews = order_reviews.groupby('order_id').mean()
order_reviews = order_reviews.reset_index() # pour remettre order_id en tant que colonne
print('Lignes après traitement :', len(order_reviews))
print('Valeurs manquantes :', order_reviews.isnull().sum().sum())

Lignes avant traitement : 99224
Lignes après traitement : 98673
Valeurs manquantes : 0


### **Traitement de *orders***

Je convertis les colonnes contenant des dates que je vais utiliser au format date.<br>
Je décide de ne conserver que les commandes qui sont indiquées comme livrées (la grande majorité)<br>
Je crée une nouvelle colonne *delivery_time* en faisant la différence entre *order_purchase_timestamp* et *order_delivered_customer_date*.<br>
Au final, je ne conserve que *order_id*, *customer_id*, *order_purchase_timestamp* et *delivery_time*



In [9]:
print('Lignes avant traitement :', len(orders))
# je passe par .apply() pour appliquer pd.to_datetime au deux colonnes
orders[['order_purchase_timestamp', 'order_delivered_customer_date']] = orders[['order_purchase_timestamp', 'order_delivered_customer_date']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')
orders = orders.loc[orders['order_status'] == 'delivered']
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders = orders[['order_id', 'customer_id', 'order_purchase_timestamp', 'delivery_time']]
# Je supprime les quelques lignes où delivery_time est manquant
orders = orders.loc[orders['delivery_time'].isnull() == False]
print('Lignes après traitement :', len(orders))
print('Valeurs manquantes :', orders.isnull().sum().sum())


Lignes avant traitement : 99441
Lignes après traitement : 96470
Valeurs manquantes : 0


### **Traitement de *products***

Je fusionne avec *product_category_name_translation* pour avoir les catégories en Anglais.<br>
Je remplace les valeurs manquantes dans *product_category_name_english* par "other".<br>
Au final, je ne conserve que les colonnes *product_id* et *product_category_name_english*.

In [10]:
products = products.merge(product_category_name_translation, on='product_category_name', how='left')
products['product_category_name_english'].loc[products['product_category_name_english'].isnull() == True] = "other"
products = products[['product_id', 'product_category_name_english']]
print('Valeurs manquantes :', products.isnull().sum().sum())

Valeurs manquantes : 0


C:\Users\Data Science\AppData\Local\Temp\ipykernel_17288\1600838163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['product_category_name_english'].loc[products['product_category_name_english'].isnull() == True] = "other"
